In [2]:
import os
from glob import glob
import pandas as pd
import html
import itertools
import pickle
import re
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.cluster import KMeans, k_means_, DBSCAN, AgglomerativeClustering
from sklearn import cluster
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import cosine_distances

import hdbscan
from gensim import models
from gensim.corpora import mmcorpus, Dictionary
from gensim.models import lsimodel, ldamodel, tfidfmodel, rpmodel, logentropy_model, TfidfModel, LsiModel, LdaModel
from gensim import matutils, corpora
from gensim.models.doc2vec import TaggedDocument
from gensim.models import doc2vec
from gensim.models.word2vec import Word2Vec
from sklearn.svm import SVC

import sys
sys.path.append('~/Documents/GitHub/Private_Project/personal_project/')
import html
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import chat_bot as cb
import Database_Handler as dh
from datetime import datetime
import seaborn as sns
import functools
from konlpy.tag import Mecab
from konlpy.utils import pprint
mecab = Mecab()
%matplotlib inline

C:\Users\pc\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
dataDict = pickle.load(open('./data/pre_data/stastics/for_statistics_daum_from_mongodb.pickled','rb'))
print (len(dataDict))

4722


In [4]:
keywordsDict = pickle.load(open('./data/pre_data/keywords/keywords_daum.pickled','rb'))

In [5]:
for idx in dataDict:
    dataDict[idx]['extracted_keywords'] = keywordsDict[idx]

In [6]:
df_daum = pd.DataFrame.from_dict(dataDict, orient='index')
df_daum['date'] = pd.to_datetime(df_daum['date']).dt.date
df_daum.reset_index(inplace = True)
df_daum.rename(columns={'index':'id'}, inplace=True)
print (df_daum.shape)
df_daum.head()

(4722, 11)


,id,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords
0,5a2a61bf588c13481c229d1e,뉴스,2017-12-07,세계일보,1093,911,1,"""밤이 무섭다""..비아그라 공장 연기에 남성들 부작용 호소","주민들은 공장에서 배출된 연기가 '남성이 매우 건강해지는 부작용'을 일으킨다며, ...","[부작용, 비아그라, 아일랜드]","{공장, 세보 효과, 남성들, 지역, 건강, 연기, 부작용}"
1,5a2a61bf588c13481c229d1f,뉴스,2017-12-07,헬스조선,603,386,2,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,점심식사를 마친 후 후식으로 커피를 마시는 사람들이 많다. 실제로 직장이 밀집돼 ...,"[커피, 낮잠, 음식물]","{철분, 입냄새, 커피, 낮잠, 자세, 디스크, 건강, 점심 식사, 식후, 치아}"
2,5a2a61bf588c13481c229d20,뉴스,2017-12-07,연합뉴스,1067,811,3,"'십년지기 생매장' 진짜 이유는..""'청부 통정' 알려질까 봐""",(성남=연합뉴스) 최해민 기자 = 십년지기 지인을 산 채로 묻어 살해한 50대 여...,"[살인혐의, 철원, 검찰송치]","{진술, 범행, 아들, 성관계, 앙심, 철원, 남편, 주변, 지인, 경찰}"
3,5a2a61bf588c13481c229d21,뉴스,2017-12-07,헤럴드경제,418,369,4,"신영자, 억 소리나는 갑질","신영자, 적용안된 혐의→검찰 상고에서 인정\n신영자, 얼마를 어떻게 받았나 [헤럴...","[신영자, 갑질, 롯데백화점]","{롯데, 징역, 검찰, 네이처리퍼블릭, 유통업체, 매장, 신영자 이사장, 혐의}"
4,5a2a61bf588c13481c229d22,뉴스,2017-12-07,연합뉴스,434,368,5,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지",(청주=연합뉴스) 이승민 기자 = 지난 9월 청주의 한 하천에서 20대 여성을 둔기...,"[공범, 살인, 과자]","{남자친구, 범행, 유치장, 폭행, 쪽지, 혐의, 경찰, 과자}"


In [7]:
def nav_tokenizer(corpus):
    pos = mecab.pos(corpus)
    res = [x[0] for x in pos if (x[1] == u'VV' or x[1] == u'VA' or x[1] == u'NNB' or x[1] == u'NNP' or x[1] == u'NNG')]
    return res

In [8]:
def remove_stopwords(words, stopwords):
    res = [x for x in words if x not in stopwords]
    res = [x for x in words if len(x) != 1]
    return res

In [9]:
stopwords = open('./data/stopwordsList.txt',encoding='utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

In [10]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

In [11]:
w2v_docs = list()
for idx in tqdm(df_daum.index):
    text = df_daum.loc[idx,'title']+'.\n'+df_daum.loc[idx,'mainText']
    pos = nav_tokenizer(text)
    pos = remove_stopwords(pos, stopwords)
    label = df_daum.loc[idx, 'category']
    w2v_docs.append(TaggedDocument(pos, label))

100%|██████████| 4722/4722 [01:35<00:00, 49.46it/s]


In [12]:
train, test = train_test_split(w2v_docs, test_size = 0.1)

In [13]:
x_train = [ x.words for x in train] 
y_train = [ x.tags for x in train] 
x_test = [ x.words for x in test] 
y_test = [ x.tags for x in test] 

### word2vec

In [14]:
w2v_model = Word2Vec(sg=1, size=3000,window=8, workers = 6, iter = 20)
w2v_model.build_vocab(tqdm(x_train))
w2v_model.train(tqdm(x_train), total_examples=w2v_model.corpus_count, epochs=w2v_model.iter)

100%|██████████| 4249/4249 [00:44<00:00, 94.77it/s] 


15310518

In [15]:
print(w2v_model.most_similar('신혜선'))

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('박시후', 0.7483072280883789), ('황금빛', 0.6838359832763672), ('내인생', 0.6692831516265869), ('서은수', 0.640835165977478), ('셰어하우스', 0.6082226037979126), ('나영희', 0.6053473353385925), ('재벌가', 0.6002784967422485), ('소현경', 0.5949802398681641), ('황금빛내인생', 0.5904324054718018), ('서지안', 0.582492470741272)]


In [16]:
print(w2v_model.most_similar('손흥민'))

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('토트넘', 0.639374852180481), ('스토크시티', 0.5924640893936157), ('왓퍼드', 0.59075927734375), ('득점포', 0.5882641077041626), ('델레', 0.5849654674530029), ('사우스햄턴', 0.5846250057220459), ('에릭센', 0.5837652683258057), ('브라이튼', 0.579013466835022), ('마우리시오', 0.5676306486129761), ('유럽챔피언스리그', 0.5655679106712341)]


In [17]:
print(w2v_model.most_similar('황금빛내인생'))

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('이다인', 0.6932629942893982), ('빅픽처', 0.67424476146698), ('김형석', 0.666618824005127), ('나영희', 0.6655270457267761), ('재벌남', 0.6579344272613525), ('셰어하우스', 0.6488503813743591), ('내인생', 0.6413261890411377), ('소현경', 0.6400668621063232), ('이진', 0.6392889022827148), ('서은수', 0.6356393694877625)]


In [18]:
print(w2v_model.most_similar('이명박'))

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('언론장악', 0.4950806498527527), ('반정부', 0.47899162769317627), ('박근혜정부', 0.4744024872779846), ('탈원전', 0.4701486825942993), ('김태효', 0.4650428891181946), ('박근혜', 0.46187707781791687), ('실소유주', 0.4520929157733917), ('정호영', 0.4490810036659241), ('노종면', 0.44388866424560547), ('정무수석', 0.44284212589263916)]


In [19]:
print(w2v_model.most_similar('문재인'))

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('대통령', 0.48957154154777527), ('조원진', 0.4687511920928955), ('문재', 0.46677735447883606), ('조찬', 0.45316943526268005), ('대한애국당', 0.44852203130722046), ('국빈방문', 0.44844523072242737), ('방중', 0.4449848532676697), ('외교부장', 0.42320626974105835), ('케어', 0.4218031167984009), ('정책실장', 0.4030939042568207)]


In [20]:
print(w2v_model.most_similar('노무현'))

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('김영삼', 0.6032348871231079), ('김대중', 0.564887523651123), ('이회창', 0.544792115688324), ('서거', 0.5171603560447693), ('박연차', 0.5082685947418213), ('장준하', 0.5071714520454407), ('안희정', 0.5023866891860962), ('현철', 0.500705361366272), ('박사모', 0.4894034266471863), ('문희상', 0.48821043968200684)]


In [21]:
print(w2v_model.most_similar('박근혜'))

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('탄핵', 0.5290473699569702), ('이명박', 0.4618770480155945), ('최순실', 0.456400990486145), ('탄핵심판', 0.44233542680740356), ('정권', 0.4398067593574524), ('박근혜정부', 0.43897557258605957), ('비선실세', 0.43826571106910706), ('지국장', 0.43732163310050964), ('친박', 0.4350965917110443), ('안봉근', 0.4348442554473877)]


In [22]:
print(w2v_model.most_similar('이국종'))

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('아주대병원', 0.8215057253837585), ('중증외상센터', 0.7649655938148499), ('귀순병', 0.6857060194015503), ('아주대', 0.6851359605789185), ('오청', 0.6824158430099487), ('문종환', 0.6522502303123474), ('총상', 0.6439065933227539), ('권준식', 0.63106369972229), ('외상센터', 0.625391960144043), ('공동경비구역', 0.6198679208755493)]


In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
print ('building tf-idf matrix ...')
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)
matrix = vectorizer.fit_transform(x_train)
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print ('vocab size :', len(tfidf))

building tf-idf matrix ...
vocab size : 7479


In [24]:
def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += w2v_model[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [25]:
from sklearn.preprocessing import scale
train_vecs_w2v = np.concatenate([buildWordVector(z, 3000) for z in tqdm(map(lambda x: x, x_train))])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([buildWordVector(z, 3000) for z in tqdm(map(lambda x: x, x_test))])
test_vecs_w2v = scale(test_vecs_w2v)

0it [00:00, ?it/s]C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
4249it [00:28, 151.44it/s]
473it [00:03, 155.05it/s]


In [26]:
w2v_model.save('./model/word2vec_category_size3000_window8_iter20_by_mecab.w2v')

In [ ]:
w2v_model = Word2Vec.load('./model/word2vec_category_size3000_window8_iter20_by_mecab.w2v')

In [29]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [31]:
le.fit(y_train)
le.classes_

array(['뉴스', '스포츠', '연예'], dtype='<U3')

In [32]:
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Embedding, embeddings, merge
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=3000))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_vecs_w2v, le.transform(y_train), epochs=100, verbose=2)

Epoch 1/100
 - 1s - loss: -4.6609e+00 - acc: 0.6451
Epoch 2/100
 - 1s - loss: -4.9325e+00 - acc: 0.6620
Epoch 3/100
 - 1s - loss: -4.9893e+00 - acc: 0.6667
Epoch 4/100
 - 1s - loss: -5.0262e+00 - acc: 0.6700
Epoch 5/100
 - 1s - loss: -5.0477e+00 - acc: 0.6712
Epoch 6/100
 - 1s - loss: -5.0535e+00 - acc: 0.6733
Epoch 7/100
 - 1s - loss: -5.0508e+00 - acc: 0.6740
Epoch 8/100
 - 1s - loss: -5.0659e+00 - acc: 0.6745
Epoch 9/100
 - 1s - loss: -5.0751e+00 - acc: 0.6747
Epoch 10/100
 - 1s - loss: -5.0746e+00 - acc: 0.6750
Epoch 11/100
 - 1s - loss: -5.0800e+00 - acc: 0.6752
Epoch 12/100
 - 1s - loss: -5.0784e+00 - acc: 0.6755
Epoch 13/100
 - 1s - loss: -5.0794e+00 - acc: 0.6752
Epoch 14/100
 - 1s - loss: -5.0871e+00 - acc: 0.6762
Epoch 15/100
 - 1s - loss: -5.0873e+00 - acc: 0.6759
Epoch 16/100
 - 0s - loss: -5.0785e+00 - acc: 0.6759
Epoch 17/100
 - 1s - loss: -5.0844e+00 - acc: 0.6757
Epoch 18/100
 - 1s - loss: -5.0847e+00 - acc: 0.6757
Epoch 19/100
 - 1s - loss: -5.0858e+00 - acc: 0.6757
Ep

In [33]:
score = model.evaluate(test_vecs_w2v, le.transform(y_test), verbose=2)
print (score[1])

0.646934460131865
